Zaimportowanie bibliotek:

In [1]:
from os import listdir
from os.path import join
import numpy as np
import skimage.io as sk_io
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score, confusion_matrix

Wczytanie próbek i etykiet, zapis gotowych plików:

In [2]:
def get_samples():
    lista = ["szafy","drzwi","drabina","lampy","krzesła","biurka"]
    X = np.zeros([1,2])
    Y = []
    for i in range(len(lista)):
        folder_path = (R'./'+lista[i]+'/')
        print(i)
        for j in range(len(listdir(folder_path))):
            element_path = join(folder_path, listdir(folder_path)[j])
            sample = np.array([(sk_io.imread(element_path))])
            Y.append(i)
            if j == 0 and i == 0:
                X = sample
            else:
                X = np.concatenate([X, sample], axis=0)
        np.save("X_lab5.npy", X)
        np.save("Y_lab5.npy", np.array(Y).reshape([-1]))
get_samples()

0
1
2
3
4
5


Wczytanie plików z danymi i etykietami:

In [15]:
X = np.load("X_lab5.npy")
Y = np.load("Y_lab5.npy")

#onehotenc = OneHotEncoder(sparse_output=False)
#Y = onehotenc.fit_transform(Y.reshape(-1, 1))

Podział próbek na treningowe i testowe:

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)

Budowa modelu i uruchomienie uczenia:

In [48]:
model = Sequential()

# Dodaj warstwy modelu
model.add(Conv2D(64, kernel_size=3, activation='relu', input_shape=(256, 256, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(32, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(6, activation='sigmoid'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=25, batch_size=100)

Epoch 1/25
8/8 [==============================] - 21s 3s/step - loss: 91.8347 - accuracy: 0.1746 - val_loss: 28.4124 - val_accuracy: 0.1759
Epoch 2/25
8/8 [==============================] - 19s 2s/step - loss: 12.8929 - accuracy: 0.1984 - val_loss: 1.9314 - val_accuracy: 0.1914
Epoch 3/25
8/8 [==============================] - 19s 2s/step - loss: 1.6209 - accuracy: 0.3201 - val_loss: 1.6272 - val_accuracy: 0.3364
Epoch 4/25
8/8 [==============================] - 19s 2s/step - loss: 1.3875 - accuracy: 0.4656 - val_loss: 1.6121 - val_accuracy: 0.3642
Epoch 5/25
8/8 [==============================] - 19s 2s/step - loss: 1.1765 - accuracy: 0.6045 - val_loss: 1.7698 - val_accuracy: 0.4599
Epoch 6/25
8/8 [==============================] - 19s 2s/step - loss: 0.8754 - accuracy: 0.6918 - val_loss: 1.8745 - val_accuracy: 0.4537
Epoch 7/25
8/8 [==============================] - 20s 3s/step - loss: 0.7707 - accuracy: 0.7526 - val_loss: 1.6740 - val_accuracy: 0.4383
Epoch 8/25
8/8 [===============

Porównanie wyników uczenia z poprawnymi wartościami zbioru testowego, wyświetlona dokładność i macierz pomyłek:

In [50]:
y_pred = np.argmax(model.predict(X_test), axis=1)
accuracy = accuracy_score(y_test,y_pred)
conf_matrix = confusion_matrix(y_test,y_pred,normalize="true")
print("model accuracy = ",accuracy)
print("confusion matrix:\n",conf_matrix)

11/11 [==============================] - 2s 170ms/step
model accuracy =  0.5462962962962963
confusion matrix:
 [[0.47368421 0.21052632 0.         0.07017544 0.07017544 0.1754386 ]
 [0.16363636 0.61818182 0.05454545 0.07272727 0.05454545 0.03636364]
 [0.05263158 0.03508772 0.61403509 0.0877193  0.10526316 0.10526316]
 [0.15217391 0.13043478 0.08695652 0.39130435 0.19565217 0.04347826]
 [0.01724138 0.06896552 0.01724138 0.10344828 0.75862069 0.03448276]
 [0.21568627 0.21568627 0.         0.03921569 0.15686275 0.37254902]]
